# 2-D Diffusion Equation (Stationary)

## Introduction

Assuming that the temperature no longer changes with time, i.e., the temperature field is stationary, the heat conduction equation simplifies to a pure diffusion equation in the form of the so-called Poisson equation (with radiogenic heat production only):

$$
0 = \frac{\partial}{\partial{x}} \left(k_{x} \frac{\partial{T}}{\partial{x}}\right) + \frac{\partial}{\partial{y}} \left(k_{y} \frac{\partial{T}}{\partial{y}}\right) + \rho H_r, \tag{1}
$$

where $\rho$ is the density, $k_{x,y}$ are the thermal conductivities in the x- and y-directions, and $H_r$ is the radiogenic heat production per unit mass [W/kg]. If we further simplify the equation and assume that the thermal parameters (especially the thermal conductivity $k$) are **isotropic** and **constant**, we obtain:

$$
0 = \left( \frac{\partial^2{T}}{\partial{x}^2} + \frac{\partial^2{T}}{\partial{y}^2} \right) + \frac{Q}{k}, \tag{2}
$$

where $Q = \rho H_r$ is the volumetric heat production rate [W/m^3].

## The Problem
We consider equation (2) in a 2D rectangular domain with width $L = 4000$ m and depth $H = 2000$ m. Within the domain, a body with high heat production is embedded, defined by the corner coordinates (x, z) in km: (1900, 900), (2100, 900), (2100, 1100), and (1900, 1100). The volumetric heat production $\rho H$ is 0.3 W/m<sup>3</sup> inside the body and 0 outside. The thermal conductivity is $k = 6.5$ W/(m·K). A Dirichlet boundary condition $T = 0$ °C is imposed along all boundaries.

<img src="./Figures/Exercise04_1.png" alt="drawing" width="450"/> <br>
**Fig. 1.** Model setup

This configuration roughly represents the situation of high-level radioactive waste disposal in a salt dome. Radioactive containers with a diameter of 1 m are stored in boreholes about 250 m deep at a depth of ~1 km. The average spacing of such boreholes is about 50 m over a width of 250 m and a length of 1–2 km. With a heat production rate of slightly less than 1 kW per 1 m high container, the average volumetric heat production in the storage area corresponds approximately to the value used above.

To solve this problem with Julia, we first need to load the required modules (`ExtendableSparse`, `Plots`) and submodules (`GeoModBox.HeatEquation.TwoD`).

In [ ]:
using ?

We now define the model dimensions ($L$, $H$) and physical parameters ($k$, $Q$):

In [ ]:
# Physical parameters --------------------------------------------------- #
P       = ( 
    L       =   ?,      # Length   [m]
    H       =   ?,      # Height   [m]
    k       =   ?,        #   Thermal conductivity, W/m/K
    # Define the region of the heat source
    Wcave   =   ?,      # Width of the region [m]
    Hcave   =   ?,      # Thickness [m]
    Dcave   =   ?,      # Depth of the center [m]
    Xcave   =   ?,      # x-position of the center [m]
    Q       =   ?         # Volumetric heat production rate [W/m³]; Q = rho*H
)
# ----------------------------------------------------------------------- #

Next, we define the number of grid points and the grid spacing:

In [ ]:
# Numerical parameters -------------------------------------------------- #
NC      = (
    x       =   ?,        # Grid points in x-direction, columns
    y       =   ?         # Grid points in y-direction, rows    
)
Δ       = (
    x       =   ?,   # Grid spacing in x-direction
    y       =   ?    # Grid spacing in y-direction
)
# ----------------------------------------------------------------------- #

With these, the numerical grid can be defined, along with the initial conditions of our problem:

In [ ]:
# Grid creation ---------------------------------------------------------- #
x       = (
    c       =   LinRange(?),
)
y       = (
    c       =   LinRange(?),
)
# ----------------------------------------------------------------------- #
# Initialization --------------------------------------------------------- #
D       = ( 
    Q       =   zeros(NC...),           # (row, col) 
    T       =   zeros(NC...),
)
# Define the anomaly region ---------------------------------------------- #
for i = 1:NC.x, j = 1:NC.y
    ?
end
# ----------------------------------------------------------------------- #

## The Solution

### Discretization

To solve the problem numerically, we need to divide our model domain into a numerical grid. We assume that the temperature is defined at so-called *cell-centered* grid points (see Fig. 2). To incorporate the boundary conditions correctly, we also use *ghost nodes* in our grid.  

#### Grid and Indexing

<img src="./Figures/Exercise04_2.png" alt="drawing" width="450"/> <br>  
**Fig. 2.** Staggered grid.  

The given staggered grid allows for a *conservative* finite-difference approximation, where the heat flux  
$q_{i,j} = -k \frac{\partial{T}}{\partial{x_{i,j}}}$ is defined at the midpoints of the grid lines and the temperature at the centers of the grid cells. (Strictly speaking, the thermal conductivity is then also defined on the grid lines; however, since it is constant, we do not need to consider this here. For variable thermal parameters, the discretization would need to be modified.)  

The use of central temperature points, in combination with *ghost nodes*, allows us to implement boundary conditions with the same order of accuracy as the central difference quotients in the interior of the model.  

When indexing grid points, we distinguish between *local* and *global* indices.  
- The local index describes the position on the *(i, j)* grid.  
- The global index is a running index from 1 to *nx* × *ny*, corresponding to the total number of equations, i.e., the number of interior grid points. The global index is also used to assemble the coefficient matrix of the linear system.  

For each grid point (i.e., for each equation), a so-called numerical *stencil* specifies the positions of the grid points relevant to that equation. The coefficients for these points are nonzero, while all others are zero. The stencil nomenclature often follows that of a compass, with points at South, West, Center, East, and North. For each equation, the global index ($ii$) of the stencil points is given by their relative position to the central stencil point:  

$$
iS = ii - nx,\\\\
iW = ii - 1,\\\\   
iZ = ii, \\\\
iE = ii + 1,\\\\
iN = ii + nx.
$$

#### Finite Difference Approximation

The partial differential equation can then be approximated by finite differences as:

$$
0 = \left( \frac{T_{i-1,j} - 2T_{i,j} + T_{i+1,j}}{\Delta{x}^2} + \frac{T_{i,j-1} - 2T_{i,j} + T_{i,j+1}}{\Delta{y}^2} \right) + \frac{Q}{k}, \tag{3}
$$

where $i, j$ are the indices and $\Delta{x}, \Delta{y}$ are the grid spacings in the x- and y-directions. Rearranging gives a linear system with five coefficients of the form:  

$$
b T_{i,j-1} + aT_{i-1,j} - (2a + 2b) T_{i,j} + a T_{i+1,j} + b T_{i,j+1} = - \frac{Q}{k}, \tag{4}
$$

where $a = 1 / \Delta{x}^2$ and $b = 1 / \Delta{y}^2$.  

### Boundary Conditions

The temperature at the *ghost nodes* is defined for *Dirichlet* and *Neumann* boundary conditions in the same way as in the 1D case of the [explicit](./02_1D_Heat_explicit.ipynb) or [implicit](./03_1D_Heat_implicit.ipynb) solutions of the heat diffusion equation (this time for four boundaries instead of two). Since we again have a linear system of equations, the coefficients and right-hand side for the **interior grid points** near the boundaries must be modified according to the boundary conditions, using the temperature at the ghost nodes as follows (derivation in lecture):  

**Dirichlet** <br>  
*West*  
$$
bT_{1,j-1} - (3a + 2b)T_{1,j} + aT_{2,j} + bT_{1,j+1} = -\frac{Q}{k} - 2aT_{BC,W} \tag{5}
$$
*East*  
$$
bT_{nx,j-1} + aT_{nx-1,j} - (3a + 2b)T_{nx,j} + bT_{nx,j+1} = -\frac{Q}{k} - 2aT_{BC,E} \tag{6}
$$
*South*  
$$
aT_{i-1,1} - (2a + 3b)T_{i,1} + aT_{i+1,1} + bT_{i,2} = -\frac{Q}{k} - 2bT_{BC,S} \tag{7}
$$
*North*  
$$
bT_{i,ny-1} + aT_{i-1,ny} - (2a + 3b)T_{i,ny} + aT_{i+1,ny} = -\frac{Q}{k} - 2bT_{BC,N} \tag{8}
$$

**Neumann** <br>  
*West*  
$$
bT_{1,j-1} - (a + 2b)T_{1,j} + aT_{2,j} + bT_{1,j+1} = -\frac{Q}{k} + a c_W \Delta{x} \tag{9}
$$
*East*  
$$
bT_{nx,j-1} + aT_{nx-1,j} - (a + 2b)T_{nx,j} + bT_{nx,j+1} = -\frac{Q}{k} - a c_E \Delta{x} \tag{10}
$$
*South*  
$$
aT_{i-1,1} - (2a + b)T_{i,1} + aT_{i+1,1} + bT_{i,2} = -\frac{Q}{k} + b c_S \Delta{y} \tag{11}
$$
*North*  
$$
bT_{i,ny-1} + aT_{i-1,ny} - (2a + b)T_{i,ny} + aT_{i+1,ny} = -\frac{Q}{k} - b c_N \Delta{y} \tag{12}
$$

When initializing the boundary conditions, we make use of a small trick and define the temperature at the *ghost nodes* later in the script.


In [ ]:
# Boundary Conditions---------------------------------------------------- #
BC      =   (
    type    = (W=:?, E=:?, N=:?, S=:?),
    val     = (W=:?,E=:?,N=:?,S=:?)
)
# ----------------------------------------------------------------------- #

### Solving the Problem

Equation (4) represents a linear system of equations of the form  

$$
\mathbf{K} T = T_i, \tag{13}
$$

with a coefficient matrix $\mathbf{K}$ containing five nonzero diagonals, the initial temperature condition $T_i$, and the stationary solution $T$. We therefore proceed by defining the parameters for our linear system:


In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   maximum(?)
K       =   ExtendableSparseMatrix(?)
rhs     =   zeros(?)
# ----------------------------------------------------------------------- #

The assembly of the coefficient matrix, the modification of the right-hand side as described above, and the solution of the linear system are carried out in the function `Poisson!()`. By importing the submodule `GeoModBox.HeatEquation.TwoD`, this function can be called directly (pay attention to the parameters passed to the function!).

In [ ]:
# Solve equation -------------------------------------------------------- #
Poisson2Dc!( ? )
# ----------------------------------------------------------------------- #

Alternatively, the computational steps from the function could be programmed directly here, in which case the submodule would no longer need to be loaded.

## Visualization

Finally, we plot the result:

In [ ]:
# Plot solution --------------------------------------------------------- #
p = heatmap( ? )

contour!( ? )

display(p)

savefig("./Results/04_Steady_State_Solution.png")
# ----------------------------------------------------------------------- #